In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/test.csv
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/train.csv

--2023-07-20 16:48:35--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89724 (88K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  87.62K  --.-KB/s    in 0.02s   

2023-07-20 16:48:35 (5.67 MB/s) - ‘test.csv’ saved [89724/89724]

--2023-07-20 16:48:35--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791227 (773K) [text/

In [ ]:
train_set = pd.read_csv("/content/train.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)

test_set = pd.read_csv("/content/test.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)


In [ ]:
train_set

,text,category
0,Trailer late ah parthavanga like podunga,Positive
1,Move pathutu vanthu trailer pakurvnga yaru,Positive
2,Puthupetai dhanush ah yarellam pathinga,Positive
3,"Dhanush oda character ,puthu sa erukay , mass ta",Positive
4,vera level ippa pesungada mokka nu thalaivaaaaaa,Positive
...,...,...
11330,Yuvan shankar Raja anna fan's like here...,Positive
11331,A masterpiece best revenge film I’ve ever scene,Positive
11332,Enna pa thala ya kamiya than katringa,Positive
11333,R A A S H I K H A N N A,unknown_state


In [ ]:
test_set

,text,category
0,Daily likes & views pakka vanthavaga ellarukum...,Positive
1,25 k dislikes ethuku da intha trailerku poi ap...,Negative
2,#Lyca unna nenacha pavama iruku ya,Mixed_feelings
3,It looks like Hindi movie amitab bachan,Positive
4,Thalaivarukku nejamavey vayasaagiduchu... siva...,Positive
...,...,...
1255,Sappypathy Vijayamma bigilu ummpi get more dis...,Negative
1256,Stylea erukana ha ha naturally,Positive
1257,Style la irukana hahaha mass dialogue,Negative
1258,Like for lokesh kanagaraj THALAPATHY 64 Director,unknown_state


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
def take_data_to_shower(tweet):
    tweet=str(tweet)

    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

In [ ]:
def remove_emoticon(text):
        text=str(text)

        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

In [ ]:
def remove_punc(text):
    text=str(text)
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
def tokenize(tweet):
   tweet=str(tweet)

   lower_tweet = tweet.lower()
   return word_tokenize(lower_tweet)

In [ ]:
def remove_stop_words(tokens):
   clean_tokens = []
   newStopWords = ['rt', 'mkr', 'didn', 'bc', 'n', 'm', 'im', 'll', 'y', 've', 'u', 'ur', 'don', 't', 's', 'co','oru', 'enru', 'matrum', 'indha', 'idhu', 'enru', 'kondu', 'enpathu', 'pala', 'aagum', 'allathu', 'avar', 'naan', 'ullu', 'andha', 'ivar', 'ena', 'muthal', 'enna', 'irundhu', 'sila', 'en', 'pontu', 'venṭum', 'vandhu', 'ithan', 'athu', 'avan', 'thaan', 'palarum', 'ennum', 'melum', 'pinar', 'kondu', 'irukkum', 'thanathu', 'ullathu', 'pothu', 'enrum', 'athan', 'than', 'piragu', 'avargal', 'varai', 'aval', 'nee', 'aagiyathu', 'ullana', 'vandha', 'irundh', 'migavum', 'inggu', 'meethu', 'or', 'ivai', 'indha', 'ithaṉaḷ', 'patri', 'varum', 'veṟu', 'iru', 'ithil', 'pol', 'ippothu', 'avarathu', 'maṭṭum', 'indha', 'enuṁ', 'mel', 'pin', 'cernta', 'ākiyōr', 'enakku', 'innum', 'andha', 'anṟu', 'oṟe', 'miga', 'angu', 'palvēru', 'viṭṭu', 'perum', 'athai', 'paṟṟiya', 'uṉ', 'athik', 'andhak', 'pēr', 'indhaṉāḷ', 'avai', 'athe', 'ēn', 'muṟai', 'yār', 'eṉappaṭum', 'ellām', 'maṭṭumē', 'ingkē', 'angkē', 'iṭam', 'iṭattil', 'athil', 'nām', 'atharku', 'eṉavē', 'piṟa', 'siṟu', 'maṟṟa', 'viṭa', 'endha', 'eṉavum', 'eṉappuṭum', 'eṉiṉum', 'aṭutta', 'ithanai', 'ithai', 'koḷḷa', 'indhath', 'ithaṉāl', 'athāl', 'thavira', 'pol', 'varail', 'caṟṟu', 'eṉak']
   stopWords = stopwords.words('english')
   stopWords.append(newStopWords)
  #  print(stopWords)

   for token in tokens:
       if token not in stopWords:
           if token.replace(' ', '') != '':
               if len(token) > 1:
                   clean_tokens.append(token)
   return clean_tokens

In [ ]:
def stem_and_lem(tokens):
   clean_tokens = []
   for token in tokens:
       token = wordnet_lemmatizer.lemmatize(token)
       token = lancaster_stemmer.stem(token)
       if len(token) > 1:
           clean_tokens.append(token)
   return ' '.join(clean_tokens)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
##EMBEDDING##
def datacleen(data):
  tqdm.pandas(desc="Remove duplicates ")
  data.drop_duplicates(subset=['text'], inplace=True)
  tqdm.pandas(desc="To lower letter")
  data = data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

  tqdm.pandas(desc="Cleaning Data Phase I...")
  data['take_data_to_shower'] = data['text'].progress_apply(take_data_to_shower)

  tqdm.pandas(desc="removing emoji...")
  data['remove_emoticon'] = data['take_data_to_shower'].progress_apply(remove_emoticon)


  tqdm.pandas(desc="removing punctuation...")
  data['remove_punc'] = data['remove_emoticon'].progress_apply(remove_punc)



  tqdm.pandas(desc="Tokenizing Data...")
  data['tokenize'] = data['remove_punc'].progress_apply(tokenize)

  tqdm.pandas(desc="Cleaning Data Phase II...")
  data['remove_stop_words'] = data['tokenize'].progress_apply(remove_stop_words)

  tqdm.pandas(desc="Stemming And Lemmatizing")
  data['stem_and_lem'] = data['remove_stop_words'].progress_apply(stem_and_lem)
  tqdm.pandas(desc="Cleaning Data Phase III...")

  data.drop(data[(data['category'] == 'unknown_state') | (data['category'] == 'non-tamil')].index, inplace=True)
  # replacement_dict = {
  #     'positive': 0,
  #     'negative': 1,
  #     'mixed_feelings':-1,
  # }
  # # Replace values in the DataFrame
  # data['category'].replace(replacement_dict, inplace=True)


  # stem_and_lem ---> outlet
  return data
  # text_vector = clean_texts['tokens'].tolist()


In [ ]:
train_set = datacleen(train_set)
train_set

Stemming And Lemmatizing: 100%|██████████| 11237/11237 [00:02<00:00, 3761.93it/s]


,text,category,take_data_to_shower,remove_emoticon,remove_punc,tokenize,remove_stop_words,stem_and_lem
0,trailer late ah parthavanga like podunga,positive,trailer late ah parthavanga like podunga,trailer late ah parthavanga like podunga,trailer late ah parthavanga like podunga,"[trailer, late, ah, parthavanga, like, podunga]","[trailer, late, ah, parthavanga, like, podunga]",trail lat ah parthavang lik podung
1,move pathutu vanthu trailer pakurvnga yaru,positive,move pathutu vanthu trailer pakurvnga yaru,move pathutu vanthu trailer pakurvnga yaru,move pathutu vanthu trailer pakurvnga yaru,"[move, pathutu, vanthu, trailer, pakurvnga, yaru]","[move, pathutu, vanthu, trailer, pakurvnga, yaru]",mov pathutu vanthu trail pakurvng yaru
2,puthupetai dhanush ah yarellam pathinga,positive,puthupetai dhanush ah yarellam pathinga,puthupetai dhanush ah yarellam pathinga,puthupetai dhanush ah yarellam pathinga,"[puthupetai, dhanush, ah, yarellam, pathinga]","[puthupetai, dhanush, ah, yarellam, pathinga]",puthupeta dhanush ah yarellam pathing
3,"dhanush oda character ,puthu sa erukay , mass ta",positive,dhanush oda character puthu sa erukay mass ta,dhanush oda character puthu sa erukay mass ta,dhanush oda character puthu sa erukay mass ta,"[dhanush, oda, character, puthu, sa, erukay, m...","[dhanush, oda, character, puthu, sa, erukay, m...",dhanush od charact puthu sa erukay mass ta
4,vera level ippa pesungada mokka nu thalaivaaaaaa,positive,vera level ippa pesungada mokka nu thalaivaaaaaa,vera level ippa pesungada mokka nu thalaivaaaaaa,vera level ippa pesungada mokka nu thalaivaaaaaa,"[vera, level, ippa, pesungada, mokka, nu, thal...","[vera, level, ippa, pesungada, mokka, nu, thal...",ver level ipp pesungad mokk nu thalaivaaaaa
...,...,...,...,...,...,...,...,...
11328,shankar anna and ar sir sema mass,positive,shankar anna and ar sir sema mass,shankar anna and ar sir sema mass,shankar anna and ar sir sema mass,"[shankar, anna, and, ar, sir, sema, mass]","[shankar, anna, ar, sir, sema, mass]",shank ann ar sir sem mass
11329,intha movie la yuvan music therikum pola thonu...,positive,intha movie la yuvan music therikum pola thonu...,intha movie la yuvan music therikum pola thonu...,intha movie la yuvan music therikum pola thonu...,"[intha, movie, la, yuvan, music, therikum, pol...","[intha, movie, la, yuvan, music, therikum, pol...",inth movy la yuv mus therik pol thonuthu enaku
11330,yuvan shankar raja anna fan's like here...,positive,yuvan shankar raja anna fan like here,yuvan shankar raja anna fan like here,yuvan shankar raja anna fan like here,"[yuvan, shankar, raja, anna, fan, like, here]","[yuvan, shankar, raja, anna, fan, like]",yuv shank raj ann fan lik
11331,a masterpiece best revenge film i’ve ever scene,positive,a masterpiece best revenge film i ve ever scene,a masterpiece best revenge film i ve ever scene,a masterpiece best revenge film i ve ever scene,"[a, masterpiece, best, revenge, film, i, ve, e...","[masterpiece, best, revenge, film, ever, scene]",masterpiec best reveng film ev scen


In [ ]:
test_set = datacleen(test_set)
test_set

Stemming And Lemmatizing: 100%|██████████| 1257/1257 [00:00<00:00, 9652.88it/s]


,text,category,take_data_to_shower,remove_emoticon,remove_punc,tokenize,remove_stop_words,stem_and_lem
0,daily likes & views pakka vanthavaga ellarukum...,positive,daily likes views pakka vanthavaga ellarukum...,daily likes views pakka vanthavaga ellarukum...,daily likes views pakka vanthavaga ellarukum...,"[daily, likes, views, pakka, vanthavaga, ellar...","[daily, likes, views, pakka, vanthavaga, ellar...",dai lik view pakk vanthavag ellaruk vanakkam
1,25 k dislikes ethuku da intha trailerku poi ap...,negative,k dislikes ethuku da intha trailerku poi ap...,k dislikes ethuku da intha trailerku poi ap...,k dislikes ethuku da intha trailerku poi ap...,"[k, dislikes, ethuku, da, intha, trailerku, po...","[dislikes, ethuku, da, intha, trailerku, poi, ...",dislik ethuku da inth trailerku poi apd ungalu...
2,#lyca unna nenacha pavama iruku ya,mixed_feelings,lyca unna nenacha pavama iruku ya,lyca unna nenacha pavama iruku ya,lyca unna nenacha pavama iruku ya,"[lyca, unna, nenacha, pavama, iruku, ya]","[lyca, unna, nenacha, pavama, iruku, ya]",lyc unn nenach pavam iruku ya
3,it looks like hindi movie amitab bachan,positive,it looks like hindi movie amitab bachan,it looks like hindi movie amitab bachan,it looks like hindi movie amitab bachan,"[it, looks, like, hindi, movie, amitab, bachan]","[looks, like, hindi, movie, amitab, bachan]",look lik hind movy amitab bach
4,thalaivarukku nejamavey vayasaagiduchu... siva...,positive,thalaivarukku nejamavey vayasaagiduchu siva...,thalaivarukku nejamavey vayasaagiduchu siva...,thalaivarukku nejamavey vayasaagiduchu siva...,"[thalaivarukku, nejamavey, vayasaagiduchu, siv...","[thalaivarukku, nejamavey, vayasaagiduchu, siv...",thalaivarukku nejamavey vayasaagiduchu sivaj t...
...,...,...,...,...,...,...,...,...
1254,earphone la u1 bgm kekum pothu vera level,positive,earphone la u bgm kekum pothu vera level,earphone la u bgm kekum pothu vera level,earphone la u bgm kekum pothu vera level,"[earphone, la, u, bgm, kekum, pothu, vera, level]","[earphone, la, bgm, kekum, pothu, vera, level]",earphon la bgm kek pothu ver level
1255,sappypathy vijayamma bigilu ummpi get more dis...,negative,sappypathy vijayamma bigilu ummpi get more dis...,sappypathy vijayamma bigilu ummpi get more dis...,sappypathy vijayamma bigilu ummpi get more dis...,"[sappypathy, vijayamma, bigilu, ummpi, get, mo...","[sappypathy, vijayamma, bigilu, ummpi, get, di...",sappypathy vijayamm bigilu ummp get dislik tar...
1256,stylea erukana ha ha naturally,positive,stylea erukana ha ha naturally,stylea erukana ha ha naturally,stylea erukana ha ha naturally,"[stylea, erukana, ha, ha, naturally]","[stylea, erukana, ha, ha, naturally]",style erukan ha ha nat
1257,style la irukana hahaha mass dialogue,negative,style la irukana hahaha mass dialogue,style la irukana hahaha mass dialogue,style la irukana hahaha mass dialogue,"[style, la, irukana, hahaha, mass, dialogue]","[style, la, irukana, hahaha, mass, dialogue]",styl la irukan hahah mass dialog


# test 1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_set['stem_and_lem'])
y_train = train_set['category']

print(X_train)
print(y_train)
print("--------------------------------------------------------------------------------")

X_test = vectorizer.transform(test_set['stem_and_lem'])
y_test = test_set['category']

print(X_test)
print(y_test)


  (0, 9514)	0.3630898392608611
  (0, 5874)	0.21228453849762285
  (0, 8947)	0.6076319950541398
  (0, 488)	0.30235215545113264
  (0, 5801)	0.5475402810620198
  (0, 12838)	0.2503342098784697
  (1, 14769)	0.3814851743958603
  (1, 8454)	0.570909891290358
  (1, 13549)	0.38804166762927866
  (1, 9100)	0.38804166762927866
  (1, 6697)	0.43366661202276463
  (1, 12838)	0.1982714193389087
  (2, 9051)	0.5134341481762337
  (2, 14756)	0.4484687250944225
  (2, 2132)	0.3146696758755511
  (2, 10207)	0.6090763985678489
  (2, 488)	0.2554801633996723
  (3, 11855)	0.3593364878707427
  (3, 6335)	0.15711044430828
  (3, 2846)	0.48232136195835357
  (3, 10550)	0.396478430368845
  (3, 10205)	0.42700052014664724
  (3, 1714)	0.3745579017107976
  (3, 7666)	0.2676732630756861
  (3, 2132)	0.24918369352704883
  :	:
  (10626, 6879)	0.32667962526124694
  (10626, 2574)	0.3921769003856386
  (10626, 9632)	0.2943231435370906
  (10626, 15021)	0.303260106784118
  (10626, 3823)	0.27490063205544163
  (10626, 5762)	0.2239697492153

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
rf_classifier = RandomForestClassifier()
gb_classifier = GradientBoostingClassifier()
svm_classifier = SVC(probability=True)

# Step 4: Train the Ensemble Model
ensemble_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('gb', gb_classifier), ('svm', svm_classifier)],
    voting='soft'  # Use 'soft' voting for probabilities-based prediction
)

ensemble_classifier.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = ensemble_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

Accuracy: 0.7392767031118587
                precision    recall  f1-score   support

mixed_feelings       1.00      0.01      0.03       141
      negative       0.52      0.10      0.16       164
     not-tamil       0.79      0.66      0.72        29
      positive       0.74      0.98      0.85       855

      accuracy                           0.74      1189
     macro avg       0.76      0.44      0.44      1189
  weighted avg       0.74      0.74      0.65      1189



## CLASSIFING##

In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
def classify(train_vectors, test_vectors, train_labels, test_labels, type="DT"):


  # Random Splitting With Ratio 3 : 1
  # train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, random_state=42, test_size=0.5)

  # Initialize Model
  classifier = None
  if(type=="MNB"):
    classifier = MultinomialNB(alpha=0.7)
    classifier.fit(train_vectors, train_labels)
  elif(type=="KNN"): #no
    classifier = KNeighborsClassifier(n_jobs=4)
    params = {'n_neighbors': [3], 'weights':['uniform', 'distance']}
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="SVM"): #no
    classifier = SVC()
    classifier = GridSearchCV(classifier, {'C':[0.001, 0.01, 0.1, 1, 10]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="DT"):
    classifier = DecisionTreeClassifier()
    #params = {'criterion':['gini','entropy']}
    #classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    #classifier = classifier.best_estimator_
  elif(type=="RF"):
    classifier = RandomForestClassifier(max_depth=800, min_samples_split=5)
    params = {'n_estimators': [n for n in range(50,200,50)], 'criterion':['gini','entropy'], }
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="LR"): #no
    classifier = LogisticRegression(multi_class='auto', solver='newton-cg')
    classifier = GridSearchCV(classifier, {"C":np.logspace(-3,3,7), "penalty":["l2"]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="XG"):
    classifier=XGBClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="AD"):
    classifier=AdaBoostClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="VE"):
    estimators = []
    model1 = LogisticRegression()
    estimators.append(('logistic', model1))
    model2 = DecisionTreeClassifier()
    estimators.append(('cart', model2))
    classifier = VotingClassifier(estimators)
    classifier.fit(train_vectors, train_labels)

  else:
    print("Wrong Classifier Type!")
    return

  accuracy = accuracy_score(train_labels, classifier.predict(train_vectors))
  print("Training Accuracy:", accuracy)
  test_predictions = classifier.predict(test_vectors)
  accuracy = accuracy_score(test_labels, test_predictions)
  print("Test Accuracy:", accuracy)
  print("Confusion Matrix:", )
  print(confusion_matrix(test_labels, test_predictions))
  # print(classification_report([i for i in test_labels],
  #                       [i for i in test_predictions]))

In [ ]:
print("\nBuilding Model SVM...")
# train_vectors, test_vectors, train_labels, test_labels
classify(X_train,X_test,y_train,y_test, "SVM") # {MNB, KNN, SVM, DT, RF, LR,XG,AD,VE}


Building Model SVM...
Training Accuracy: 0.9958607714016933
Test Accuracy: 0.7308662741799832
Confusion Matrix:
[[  8   5   2 126]
 [ 10  31   2 121]
 [  0   3  17   9]
 [ 17  22   3 813]]


In [ ]:
print("\nBuilding Model MNB...")
classify(X_train,X_test,y_train,y_test, "MNB")


Building Model MNB...
Training Accuracy: 0.7500470366886172
Test Accuracy: 0.7258200168208578
Confusion Matrix:
[[  0   0   0 141]
 [  0   3   0 161]
 [  0   1   5  23]
 [  0   0   0 855]]


In [ ]:
print("\nBuilding Model KNN...")
classify(X_train,X_test,y_train,y_test, "KNN")



Building Model KNN...
Training Accuracy: 0.9959548447789276
Test Accuracy: 0.6989066442388562
Confusion Matrix:
[[  7   3   1 130]
 [  6   5   0 153]
 [  0   0   4  25]
 [ 19  16   5 815]]


In [ ]:
print("\nBuilding Model DT...")
classify(X_train,X_test,y_train,y_test, "DT")



Building Model DT...
Training Accuracy: 0.9968015051740358
Test Accuracy: 0.6366694701429773
Confusion Matrix:
[[ 18  18   6  99]
 [ 21  31   8 104]
 [  0   0  20   9]
 [ 72  71  24 688]]


In [ ]:
print("\nBuilding Model RF...")
classify(X_train,X_test,y_train,y_test, "RF")


Building Model RF...
Training Accuracy: 0.98137347130762
Test Accuracy: 0.7266610597140454
Confusion Matrix:
[[  5   6   3 127]
 [  1  14   2 147]
 [  0   0  13  16]
 [  7  13   3 832]]


In [ ]:
print("\nBuilding Model LR...")
classify(X_train,X_test,y_train,y_test, "LR")


Building Model LR...
Training Accuracy: 0.7769520225776105
Test Accuracy: 0.7333894028595458
Confusion Matrix:
[[  1   4   2 134]
 [  3  24   0 137]
 [  0   1  11  17]
 [  3  14   2 836]]


In [ ]:
print("\nBuilding Model AD...")
classify(X_train,X_test,y_train,y_test, "AD")


Building Model AD...
Training Accuracy: 0.720131702728128
Test Accuracy: 0.7232968881412952
Confusion Matrix:
[[  0   2   1 138]
 [  1   3   1 159]
 [  0   0  12  17]
 [  2   2   6 845]]


In [ ]:
print("\nBuilding Model VE...")
classify(X_train,X_test,y_train,y_test, "VE")


Building Model VE...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Accuracy: 0.9921919096895578
Test Accuracy: 0.6425567703952901
Confusion Matrix:
[[ 20  16   6  99]
 [ 27  39   9  89]
 [  1   1  21   6]
 [ 75  77  19 684]]


In [ ]:
print("\nBuilding Model XGB...")
classify(X_train,X_test,y_train,y_test, "XG")